In [1]:
import yfinance as yf
import pandas as pd
from finta import TA

In [28]:
def create_ta_features(ticker='^GSPC', start_='2010-01-01', end_='2022-12-31', interval_='1d', fillna=True, scale_to_std=False):
    """
    Creates technical analysis features for a given ticker
    :param fillna:
    :param interval_:
    :param end_:
    :param start_:
    :param ticker: ticker symbol
    :return: dataframe with technical analysis features
    """
    # download data
    df = yf.download(ticker, start_, end_, interval=interval_)
    # rename columns
    df.rename(columns={"Open": "open", "Adj Close": "close", "High": "high", "Low": "low", "Volume": "volume"}, inplace=True)
    # drop close column
    df.drop("Close", inplace=True, axis=1)
    # get all functions in finta
    finta_functions = [func for func in dir(TA) if callable(getattr(TA, func)) and not func.startswith("__")]
    # loop through all functions in finta and append the results to the dataframe
    # skip functions that throw errors
    for func in finta_functions:
        try:
            df[func] = getattr(TA, func)(df)
        except:
            pass
    # fill in missing values
    if fillna:
        df.fillna(method='bfill', inplace=True)
        df.fillna(method='ffill', inplace=True)
    # scale to standard deviation, by column
    if scale_to_std:
        df = (df - df.mean()) / df.std()
    return df

In [30]:
create_ta_features(scale_to_std=True).head(20)

[*********************100%***********************]  1 of 1 completed


C:\Users\Pawel\PycharmProjects\FinDL\venv\lib\site-packages\finta\finta.py:399: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for x, y in zip(x.fillna(0).iteritems(), y.iteritems()):
C:\Users\Pawel\PycharmProjects\FinDL\venv\lib\site-packages\finta\finta.py:399: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for x, y in zip(x.fillna(0).iteritems(), y.iteritems()):
C:\Users\Pawel\PycharmProjects\FinDL\venv\lib\site-packages\finta\finta.py:399: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for x, y in zip(x.fillna(0).iteritems(), y.iteritems()):
C:\Users\Pawel\PycharmProjects\FinDL\venv\lib\site-packages\finta\finta.py:399: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for x, y in zip(x.fillna(0).iteritems(), y.iteritems()):
C:\Users\Pawel\PycharmProjects\FinDL\venv\li

,open,high,low,close,volume,ADL,ADX,AO,ATR,BBWIDTH,...,VAMA,VBM,VFI,VPT,VWAP,VZO,WILLIAMS,WMA,WOBV,ZLEMA
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,-1.298686,-1.286505,-1.293132,-1.282682,0.057434,-1.916319,7.381934,-0.234498,-0.653191,0.683799,...,-1.286692,0.747361,-1.050971,-1.895904,-1.302407,-0.295221,-1.980814,-1.271013,-1.099913,-1.313648
2010-01-05,-1.282409,-1.283732,-1.279795,-1.279113,-1.483494,-1.910779,7.381934,-0.234498,-0.653191,0.683799,...,-1.286692,0.747361,-1.050971,-1.889920,-1.295902,1.288806,-1.980814,-1.271013,-1.099913,-1.313648
2010-01-06,-1.279326,-1.281159,-1.275428,-1.278486,1.065213,-1.908295,7.381934,-0.234498,-0.653191,0.683799,...,-1.286692,0.747361,-1.050971,-1.884033,-1.288545,2.368653,-1.980814,-1.271013,-1.098161,-1.313648
2010-01-07,-1.278759,-1.277874,-1.278105,-1.273885,1.371287,-1.897865,7.381934,-0.234498,-0.653191,0.683799,...,-1.286692,0.747361,-1.050971,-1.872910,-1.284095,2.788762,-1.980814,-1.271013,-1.084537,-1.313648
2010-01-08,-1.274463,-1.274930,-1.273117,-1.270558,0.466385,-1.888687,7.381934,-0.234498,-0.653191,0.683799,...,-1.286692,0.747361,-1.050971,-1.863650,-1.280066,2.974177,-1.980814,-1.271013,-1.076332,-1.313648
2010-01-11,-1.268963,-1.270559,-1.267212,-1.268536,0.328959,-1.885902,7.381934,-0.234498,-0.653191,0.683799,...,-1.286692,0.747361,-1.050971,-1.861210,-1.275712,3.091776,-1.980814,-1.271013,-1.071497,-1.313648
2010-01-12,-1.271137,-1.276517,-1.277647,-1.279416,0.801781,-1.888726,7.381934,-0.234498,-0.653191,0.683799,...,-1.286692,0.747361,-1.050971,-1.874107,-1.276012,1.469591,-1.980814,-1.271013,-1.100326,-1.313648
2010-01-13,-1.277708,-1.271906,-1.276212,-1.269850,0.241230,-1.882573,7.381934,-0.234498,-0.653191,0.683799,...,-1.286692,0.747361,-1.050971,-1.864159,-1.274566,1.848202,-1.980814,-1.271013,-1.077913,-1.313648
2010-01-14,-1.269246,-1.269886,-1.265400,-1.267039,-0.020825,-1.878887,7.381934,-0.234498,-0.653191,0.683799,...,-1.286692,0.747361,-1.050971,-1.857016,-1.272115,2.125158,-1.980814,-1.271013,-1.071730,-1.313648
